In [2]:
import torch

print("Torch 버전:", torch.__version__)
print("CUDA 사용 가능 여부:", torch.cuda.is_available())
print("CUDA 버전:", torch.version.cuda)
print("cuDNN 버전:", torch.backends.cudnn.version())
print("GPU 개수:", torch.cuda.device_count())

if torch.cuda.is_available():
    print("GPU 이름:", torch.cuda.get_device_name(0))
    print("현재 사용 중인 장치:", torch.cuda.current_device())

Torch 버전: 2.6.0+cu118
CUDA 사용 가능 여부: True
CUDA 버전: 11.8
cuDNN 버전: 90100
GPU 개수: 1
GPU 이름: NVIDIA GeForce RTX 2070 with Max-Q Design
현재 사용 중인 장치: 0


In [1]:
from huggingface_hub import snapshot_download

snapshot_download(
    repo_id="deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
    local_dir="./models/DeepSeek-R1-Distill-Qwen-1.5B",
    local_dir_use_symlinks=False
)

c:\Users\f1soft\anaconda3\envs\python-39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\f1soft\anaconda3\envs\python-39\lib\site-packages\huggingface_hub\file_download.py:832: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(
Fetching 9 files: 100%|██████████| 9/9 [01:14<00:00,  8.30s/it]


'C:\\Users\\f1soft\\Desktop\\f1soft\\python\\development-Repository\\vLLM\\models\\DeepSeek-R1-Distill-Qwen-1.5B'

In [ ]:
from huggingface_hub import snapshot_download

snapshot_download(
    repo_id="deepseek-ai/deepseek-llm-7b-chat",
    local_dir="./models/deepseek-llm-7b-chat",
    local_dir_use_symlinks=False
)

In [92]:
from huggingface_hub import snapshot_download

snapshot_download(
    repo_id="kwoncho/Llama-3.2-3B-KO-EN-Translation",
    local_dir="./models/Llama-3.2-3B-KO-EN-Translation",
    local_dir_use_symlinks=False
)

c:\Users\f1soft\anaconda3\envs\python-39\lib\site-packages\huggingface_hub\file_download.py:832: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(
Fetching 10 files: 100%|██████████| 10/10 [00:16<00:00,  1.64s/it]


'C:\\Users\\f1soft\\Desktop\\f1soft\\python\\development-Repository\\vLLM\\models\\Llama-3.2-3B-KO-EN-Translation'

In [20]:
def extract_content_after_think(text):
    if "</think>" in text:
        return text.split("</think>")[-1].strip()
    return text.strip()

In [ ]:
import requests

def generate_quiz_from_subject(prompt):
    url = "http://localhost:8000/v1/chat/completions"
    payload = {
        "model": "/model",
        "messages": [
            {
                "role": "system",
                "content": (
                    "You are an AI specialized in creating quiz questions. "
                    "When the user provides a subject, generate one high-quality multiple-choice question related to that subject. "
                    "Provide exactly five answer choices labeled A to E, and clearly indicate the correct answer at the end. "
                    "Also, provide a brief explanation of why the correct answer is right. "
                    "The question should be appropriate for general learners. "
                    "Format your response as follows:\n\n"
                    "Subject: [subject]\n"
                    "Question: [your question]\n"
                    "A. [option A]\n"
                    "B. [option B]\n"
                    "C. [option C]\n"
                    "D. [option D]\n"
                    "E. [option E]\n"
                    "Answer: [Correct option letter]\n"
                    "Explanation: [Why the answer is correct]"
                )
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        "temperature": 0.1,
        "max_tokens": 1024
    }

    try:
        response = requests.post(url, json=payload)
        response.raise_for_status()
        result = response.json()

        content = result["choices"][0]["message"]["content"]
        clean_content = extract_content_after_think(content)
        
        return clean_content
    
    except requests.exceptions.RequestException as e:
        print("요청 실패:", e)

In [106]:
# model_path = "./models/Llama-3.2-3B-KO-EN-Translation"
model_path = "kwoncho/Llama-3.2-3B-KO-EN-Translation"

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16,
    device_map="cuda:0",
    )

tokenizer = AutoTokenizer.from_pretrained(model_path)

Loading checkpoint shards: 100%|██████████| 2/2 [00:21<00:00, 10.59s/it]


In [97]:
import re

def clean_text(raw: str) -> str:
    # 1. HTML 태그 제거
    text = re.sub(r'<.*?>', '', raw)

    # 2. '\\n' → 실제 줄바꿈
    text = text.replace('\\n', '\n\n')

    # 3. table 포함된 줄 제거
    text = '\n'.join(line for line in text.splitlines() if 'table' not in line.lower())

    # 4. '*' 문자 제거
    text = text.replace('*', '')

    # 5. 여러 공백 → 하나의 공백
    text = re.sub(r'[ \t]+', ' ', text)

    # 6. 보기 항목 줄바꿈 (A. ~ E.)
    text = re.sub(r'\s?A\.', r'\nA.', text)
    text = re.sub(r'\s?B\.', r'\nB.', text)
    text = re.sub(r'\s?C\.', r'\nC.', text)
    text = re.sub(r'\s?D\.', r'\nD.', text)
    text = re.sub(r'\s?E\.', r'\nE.', text)

    # 7. 연속 줄바꿈 정리
    text = re.sub(r'\n{2,}', '\n\n', text)

    # 8. 양쪽 공백 제거
    text = text.strip()

    return text

In [ ]:
response = generate_quiz_from_subject("Python")
response

"**Question:** What is a key feature of Python's object-oriented programming model?\n\n**A.** Supports recursion  \n**B.** Includes built-in libraries  \n**C.** Emphasizes strong support for object-oriented programming  \n**D.** Is used for web development  \n**E.** A simple programming language  \n\n**Answer:** C. Emphasizes strong support for object-oriented programming  \n\n**Explanation:** Python's object-oriented programming model is a fundamental aspect of the language, making it widely used in software development. This feature allows for better structure and organization of code, which is a key distinction from other programming languages."

In [121]:
prompt = \
    f"""
    ### Instruction:
    주어진 텍스트를 한국어로 번역하세요.
    ### Input:
    '''{response}'''
    ### Output:
    """

In [130]:
input_ids = tokenizer.encode(prompt, return_tensors="pt").to(model.device)
output = model.generate(
    input_ids,
    max_new_tokens=200,
    temperature=0.1,
    top_p=.9,
    repetition_penalty=1.3,
    do_sample=True,
    eos_token_id=tokenizer.eos_token_id
    )
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


In [131]:
raw_string = generated_text.split("Output:")[1]
response = clean_text(raw_string)

print(response)

Q: 파이썬의 객체지향 프로그래밍 모델에서 중요한 특징은 무엇인가?

A. 리CURSION을 지원하는 것 (Recursive functions)

B. 내장 라이브러리(Internal library)가 포함되어 있다.

C. 강력한 객체 지향 프로그래밍에 대한 지원을 중시한다.

D. 웹 개발용으로 사용된다.

E. 단순한 프로그램 언어(Simple programing languauge)
 ANSWER:
C. 강력한 객체 지향 프로그래밍을 중시함
 EXPLANATION: 파이썬의 객체지향프로그래밍모델은 소프트웨어개발 에서 널리 쓰이는 주요특성이며 이 기능은 다른 프 로그램링구어들과 비교하여 코드 구조 및 조직화 측면에서 큰 차이를 가져
